# Домашнее задание №4

1. Используя сайт HuggingFace, выберите задачу по обработке текстов на русском языке (например, классификация текста, генерация текста, ответы на вопросы и т.д.).


2. Найдите подходящий датасет для выбранной задачи, используя ресурсы Kaggle или HuggingFace. В качестве альтернативы можно написать вручную или сгенерировать небольшой тестовый датасет. Данные вам будут нужны не для обучения моделей, а для тестирования.


3. На HuggingFace выберите 3-5 различных предобученных моделей, которые могут быть применены для решения вашей задачи. Это могут быть как модели для русского языка, так и мультиязычные.


4. Создайте ноутбук на Colab или Kaggle, в котором продемонстрируете применение каждой из выбранных моделей для решения вашей задачи.


5. Проанализируйте и сравните результаты, полученные с помощью разных моделей.


6. На основе проведенного анализа, выберите наиболее подходящую модель для решения вашей задачи. Обоснуйте свой выбор.

Для реализациия была выбрана задача суммаризации текстов.
Тестовый датасет - https://www.kaggle.com/datasets/phoenix120/gazeta-summaries/data
Модели для оценки:
1. bartowski/Starling-LM-7B-beta-GGUF - квантизация 4Q_XS
3. oblivious/ruGPT-3.5-13B-GGUF - Квантизация 4Q
4. IlyaGusev/saiga_llama3_8b_gguf - Квантизация 4Q


**Импорт библиотек**

In [1]:
from llama_cpp import Llama
import re
import pandas as pd
import json
from datasets import load_dataset
import random
from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu


import warnings
warnings.filterwarnings('ignore')

**Функция для чтения датасета(он в формате json строк)**

In [2]:
def read_gazeta_records(file_name, shuffle=False, sort_by_date=True):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r", encoding="utf-8-sig") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle(records)
    return records

**Инференс GGUF моделей**

In [3]:
def model_inference(model_path, text):
    llm = Llama(
    model_path=model_path,
    n_ctx=4096,  # Context length to use
    n_threads=4,  # Number of CPU threads to use
    n_gpu_layers=10, # Number of model layers to offload to GPU
    n_predict = -1,
    temp = 0.8
    )
    generation_kwargs = {
        "max_tokens": -1,
        "stop":["<|end_of_turn|>","GPT4 Correct User:","GPT4 Correct Assistant:","<|end_of_turn|>GPT4 Correct User:"],
        "echo":True, # Echo the prompt in the output
        "top_k":40, # This is essentially greedy decoding, since the model will always return the highest-probability token. Set this value > 1 for sampling decoding
        "repeat_penalty": 1.1,
        "min_p": 0.05,
        "top_p": 0.95,    
    }

    prompt = "GPT4 Correct User: Кратко изложи текст на Русском языке: " + text + "<|end_of_turn|>GPT4 Correct Assistant:"
    res = llm(prompt, **generation_kwargs) # Res is a dictionary

    summ_text = res["choices"][0]["text"]
    summ_text = re.sub(".*(?=GPT4 Correct Assistant:)", "", summ_text)
    summ_text = re.sub("GPT4 Correct Assistant: ", "", summ_text)
    return summ_text

**Небольшая предобработка текста для вычисления метрик**

In [11]:
def clean_text(texts):
    for text in texts:
        text = text.lower()
        text = re.sub(r'[^а-яА-Я]', ' ', text)
        text = re.sub(r'ё', 'е', text)
        text = text.strip()
    return texts

**Подсчет метрик ROGUE-1, ROGUE-2, ROGUE-L - Случай Униграмм, биграмм и наиболее больших sequence'ов.**

In [5]:
def evaluate_metrics(hypothesises, references):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesises, references, avg=True)
    return scores

In [6]:
model_1_path = r"C:\Users\neytr\Python Scripts\Notebooks\GGUF_models\Starling-LM-7B-beta-IQ4_XS.gguf"
model_2_path = r"C:\Users\neytr\3. Binary Classification\model-q4_K.gguf"
model_3_path = r"C:\Users\neytr\3. Binary Classification\ruGPT-3.5-13B-Q4_0.gguf"

In [7]:
test_records = read_gazeta_records(r"C:\Users\neytr\3. Binary Classification\gazeta_test.jsonl", shuffle=True, sort_by_date=False)[: 5]
test_records

[{'date': '2020-09-02 14:05:31',
  'url': 'https://www.gazeta.ru/tech/2020/09/02/13225772/bad_bluetooth.shtml',
  'summary': 'Активированный в смартфоне Bluetooth может представлять угрозу для владельца устройства — хакеры могут использовать его, чтобы без особых усилий взломать гаджет, предупреждает эксперт. Эта технология, являющаяся «проклятием» для ИБ-специалистов, действительно обладает слабым уровнем защищенности, поэтому использовать ее рекомендуется как можно реже.',
  'title': 'Названа опасность постоянно включенного Bluetooth ',
  'text': 'Постоянно включенный Bluetooth на смартфоне грозит не только быстрой разрядкой аккумулятора, но и более серьезными проблемами, сообщил агентству « Прайм » доцент кафедры информатики РЭУ им. Плеханова Александр Тимофеев. По словам эксперта, хакеры могут использовать Bluetooth для взлома электронного устройства. «Возможность взлома Bluetooth может подвергнуть опасности любую информацию, хранящуюся на устройстве (фотографии, электронные письма

**Подсчет для Starling**

In [12]:
for i in range(len(test_records)):
    hyps = []
    hyps.append(model_inference(model_1_path, test_records[i]["text"]))
    refs = []
    refs.append(test_records[i]["summary"])
hyps = clean_text(hyps)
refs = clean_text(refs)
result = evaluate_metrics(hyps, refs)

In [13]:
result

{'rouge-1': {'r': 0.2727272727272727,
  'p': 0.11904761904761904,
  'f': 0.16574585212295118},
 'rouge-2': {'r': 0.07017543859649122,
  'p': 0.027586206896551724,
  'f': 0.0396039563449666},
 'rouge-l': {'r': 0.23636363636363636,
  'p': 0.10317460317460317,
  'f': 0.14364640460913902}}

**Подсчет для Saiga**

In [14]:
for i in range(len(test_records)):
    hyps = []
    hyps.append(model_inference(model_2_path, test_records[i]["text"]))
    refs = []
    refs.append(test_records[i]["summary"])
hyps = clean_text(hyps)
refs = clean_text(refs)
result = evaluate_metrics(hyps, refs)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from C:\Users\neytr\3. Binary Classification\model-q4_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = saiga_llama3_8b
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.

llama_model_loader: - kv   1:                               general.name str              = saiga_llama3_8b
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.head_count_kv u32              = 8
llama_model_loader: - kv   8:                       llama.rope.freq_base f32              = 500000.000000
llama_model_loader: - kv   9:     llama.attention.layer_norm_rms_epsilon f32              = 0.000010
llama_model_loader: - kv  10:                          general.file_type u32              = 15
llama_model_lo

llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.head_count_kv u32              = 8
llama_model_loader: - kv   8:                       llama.rope.freq_base f32              = 500000.000000
llama_model_loader: - kv   9:     llama.attention.layer_norm_rms_epsilon f32              = 0.000010
llama_model_loader: - kv  10:                          general.file_type u32              = 15
llama_model_loader: - kv  11:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv  12:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv  13:                       tokenizer.ggml.model str              = gpt2
llama_model_loader: - 

llama_model_loader: - kv   7:              llama.attention.head_count_kv u32              = 8
llama_model_loader: - kv   8:                       llama.rope.freq_base f32              = 500000.000000
llama_model_loader: - kv   9:     llama.attention.layer_norm_rms_epsilon f32              = 0.000010
llama_model_loader: - kv  10:                          general.file_type u32              = 15
llama_model_loader: - kv  11:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv  12:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv  13:                       tokenizer.ggml.model str              = gpt2
llama_model_loader: - kv  14:                         tokenizer.ggml.pre str              = llama-bpe
llama_model_loader: - kv  15:                      tokenizer.ggml.tokens arr[str,128256]  = ["!", "\"", "#", "$", "%", "&", "'", ...
llama_model_loader: - kv  16:                  tokenizer.ggml.token_type arr[

llama_model_loader: - kv  10:                          general.file_type u32              = 15
llama_model_loader: - kv  11:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv  12:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv  13:                       tokenizer.ggml.model str              = gpt2
llama_model_loader: - kv  14:                         tokenizer.ggml.pre str              = llama-bpe
llama_model_loader: - kv  15:                      tokenizer.ggml.tokens arr[str,128256]  = ["!", "\"", "#", "$", "%", "&", "'", ...
llama_model_loader: - kv  16:                  tokenizer.ggml.token_type arr[i32,128256]  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  17:                      tokenizer.ggml.merges arr[str,280147]  = ["Ġ Ġ", "Ġ ĠĠĠ", "ĠĠ ĠĠ", "...
llama_model_loader: - kv  18:                tokenizer.ggml.bos_token_id u32              = 128000
llama_model_loader: - k

In [15]:
result

{'rouge-1': {'r': 0.32727272727272727,
  'p': 0.09782608695652174,
  'f': 0.15062761151940626},
 'rouge-2': {'r': 0.07017543859649122,
  'p': 0.017699115044247787,
  'f': 0.02826854801982831},
 'rouge-l': {'r': 0.3090909090909091,
  'p': 0.09239130434782608,
  'f': 0.14225941068258618}}

**Подсчет для RuGPT3.5**

In [16]:
for i in range(len(test_records)):
    hyps = []
    hyps.append(model_inference(model_3_path, test_records[i]["text"]))
    refs = []
    refs.append(test_records[i]["summary"])
hyps = clean_text(hyps)
refs = clean_text(refs)
result = evaluate_metrics(hyps, refs)

llama_model_loader: loaded meta data with 18 key-value pairs and 485 tensors from C:\Users\neytr\3. Binary Classification\ruGPT-3.5-13B-Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gpt2
llama_model_loader: - kv   1:                               general.name str              = ruGPT-3.5-13B
llama_model_loader: - kv   2:                           gpt2.block_count u32              = 40
llama_model_loader: - kv   3:                        gpt2.context_length u32              = 2048
llama_model_loader: - kv   4:                      gpt2.embedding_length u32              = 5120
llama_model_loader: - kv   5:                   gpt2.feed_forward_length u32              = 20480
llama_model_loader: - kv   6:                  gpt2.attention.head_count u32              = 40
llama_model_loader: - kv   7:          gpt2.

llama_model_loader: - kv   9:                       tokenizer.ggml.model str              = gpt2
llama_model_loader: - kv  10:                      tokenizer.ggml.tokens arr[str,50272]   = ["<pad>", "<|endoftext|>", "<s>", "</...
llama_model_loader: - kv  11:                  tokenizer.ggml.token_type arr[i32,50272]   = [3, 3, 3, 3, 1, 3, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  12:                      tokenizer.ggml.merges arr[str,49995]   = ["Ġ Ð", "Ð ¾", "Ð µ", "Ð °", ...
llama_model_loader: - kv  13:                tokenizer.ggml.bos_token_id u32              = 2
llama_model_loader: - kv  14:                tokenizer.ggml.eos_token_id u32              = 3
llama_model_loader: - kv  15:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  16:               tokenizer.ggml.add_bos_token bool             = false
llama_model_loader: - kv  17:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:  322 

llm_load_vocab: missing pre-tokenizer type, using: 'default'
llm_load_vocab:                                             
llm_load_vocab: ************************************        
llm_load_vocab: GENERATION QUALITY WILL BE DEGRADED!        
llm_load_vocab: CONSIDER REGENERATING THE MODEL             
llm_load_vocab: ************************************        
llm_load_vocab:                                             
llm_load_vocab: mismatch in special tokens definition ( 21/50272 vs 20/50272 ).
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = gpt2
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 50272
llm_load_print_meta: n_merges         = 49995
llm_load_print_meta: n_ctx_train      = 2048
llm_load_print_meta: n_embd           = 5120
llm_load_print_meta: n_head           = 40
llm_load_print_meta: n_head_kv        = 40
llm_load_print_meta: n_layer          = 40
llm_load_print_meta: n_rot          

llm_load_print_meta: n_gqa            = 1
llm_load_print_meta: n_embd_k_gqa     = 5120
llm_load_print_meta: n_embd_v_gqa     = 5120
llm_load_print_meta: f_norm_eps       = 1.0e-05
llm_load_print_meta: f_norm_rms_eps   = 0.0e+00
llm_load_print_meta: f_clamp_kqv      = 0.0e+00
llm_load_print_meta: f_max_alibi_bias = 0.0e+00
llm_load_print_meta: f_logit_scale    = 0.0e+00
llm_load_print_meta: n_ff             = 20480
llm_load_print_meta: n_expert         = 0
llm_load_print_meta: n_expert_used    = 0
llm_load_print_meta: causal attn      = 1
llm_load_print_meta: pooling type     = 0
llm_load_print_meta: rope type        = -1
llm_load_print_meta: rope scaling     = linear
llm_load_print_meta: freq_base_train  = 10000.0
llm_load_print_meta: freq_scale_train = 1
llm_load_print_meta: n_yarn_orig_ctx  = 2048
llm_load_print_meta: rope_finetuned   = unknown
llm_load_print_meta: ssm_d_conv       = 0
llm_load_print_meta: ssm_d_inner      = 0
llm_load_print_meta: ssm_d_state      = 0
llm_load_print_

llm_load_print_meta: model params     = 13.11 B
llm_load_print_meta: model size       = 6.94 GiB (4.55 BPW) 
llm_load_print_meta: general.name     = ruGPT-3.5-13B
llm_load_print_meta: BOS token        = 2 '<s>'
llm_load_print_meta: EOS token        = 3 '</s>'
llm_load_print_meta: PAD token        = 0 '<pad>'
llm_load_print_meta: LF token         = 134 'Ä'
llm_load_print_meta: EOT token        = 1 '<|endoftext|>'
llm_load_tensors: ggml ctx size =    0.23 MiB
llm_load_tensors:        CPU buffer size =  7105.26 MiB
...................................................................................................
llama_new_context_with_model: n_ctx      = 4096
llama_new_context_with_model: n_batch    = 512
llama_new_context_with_model: n_ubatch   = 512
llama_new_context_with_model: flash_attn = 0
llama_new_context_with_model: freq_base  = 10000.0
llama_new_context_with_model: freq_scale = 1
llama_kv_cache_init:        CPU KV buffer size =  3200.00 MiB
llama_new_context_with_model: KV self

In [17]:
result

{'rouge-1': {'r': 0.6545454545454545,
  'p': 0.09944751381215469,
  'f': 0.17266186821363058},
 'rouge-2': {'r': 0.2807017543859649,
  'p': 0.033402922755741124,
  'f': 0.05970149063662569},
 'rouge-l': {'r': 0.6, 'p': 0.09116022099447514, 'f': 0.15827337900499747}}

**Результат**

Все три модели показывают не лучший результат, но наиболее хороший результат, на удивление у ruGPT3.5.

1. ruGPT3.5
2. Starling
3. Saiga

Для более точного определения следует еще сделать:
1. Лемматизация/стемминг текста для более точно сравнения слов в гипотезе и референсе, а так же избавление текста от шума.
2. Подсчет F1-score'а - Для этого помимо ROGUE score'а (Recall), следует еще посчитать BLEU (Precision) и потом по формуле F1 = 2 * (Bleu * Rouge) / (Bleu + Rouge) расчитать F1-score (возможно я не совсем разобрался в библиотеке rogue и там уже посчитана F1-score в 'F').
3. Использовать полный датасет, а не такую маленькую выборку.
4. Подобрать более хороший промт, добавить системный промт.